In [3]:
import { CsvParseStream } from "jsr:@std/csv";

interface SysMetrics {
  date: Date,
  available: number;
buffers: number;
cached: number;
free: number;
swapFree: number;
swapTotal: number;
total: number;
avg1min: number;
avg5min: number;
avg15min: number;
}

type SysMetricsCsvRow = {
  [K in keyof SysMetrics]: string;
}

function readOutfile(readable: ReadableStream<Uint8Array>) {
  return readable
    .pipeThrough(new TextDecoderStream())
    .pipeThrough(new CsvParseStream({
      separator: ";",
      skipFirstRow: true,
      columns: [
        "date",
        "available",
        "buffers",
        "cached",
        "free",
        "swapFree",
        "swapTotal",
        "total",
        "avg1min",
        "avg5min",
        "avg15min",
      ],
    }))
    .pipeThrough(new TransformStream<SysMetricsCsvRow, SysMetrics>({
      transform: (chunk, controller) => {
        controller.enqueue({
          available: parseInt(chunk.available),
          date: new Date(parseInt(chunk.date)),
          avg15min: parseFloat(chunk.avg15min),
          avg1min: parseFloat(chunk.avg1min),
          avg5min: parseFloat(chunk.avg5min),
          buffers: parseInt(chunk.buffers),
          cached: parseInt(chunk.cached),
          free: parseInt(chunk.free),
          swapFree: parseInt(chunk.swapFree),
          swapTotal: parseInt(chunk.swapTotal),
          total: parseInt(chunk.total),
        })
      }
    }))
}

using infile = await Deno.open("deno-sys.csv")
const sysData = await Array.fromAsync(readOutfile(infile.readable));


In [25]:
const LOCALE = new Intl.Locale("fr-FR");

const byteFormat = new Intl.NumberFormat(LOCALE, {
  notation: "compact",
  style: "unit",
  unit: "byte",
});

const avgFormat = new Intl.NumberFormat(LOCALE, {
  notation: "compact",
  style: "percent",
})

console.table(sysData.map<SysMetricsCsvRow>(v => ({
  date: v.date.toLocaleString(),
  available: byteFormat.format(v.available),
  buffers: byteFormat.format(v.buffers),
  cached: byteFormat.format(v.cached),
  free: byteFormat.format(v.free),
  swapFree: byteFormat.format(v.swapFree),
  swapTotal: byteFormat.format(v.swapTotal),
  total: byteFormat.format(v.total),
  avg15min: avgFormat.format(v.avg15min),
  avg5min: avgFormat.format(v.avg5min),
  avg1min: avgFormat.format(v.avg1min),
})));

┌───────┬───────────────────────┬────────────┬───────────┬────────┬───────────┬────────────┬────────────┬──────────┬──────────┬─────────┬─────────┐
│ (idx) │ date                  │ available  │ buffers   │ cached │ free      │ swapFree   │ swapTotal  │ total    │ avg15min │ avg5min │ avg1min │
├───────┼───────────────────────┼────────────┼───────────┼────────┼───────────┼────────────┼────────────┼──────────┼──────────┼─────────┼─────────┤
│     0 │ "25/07/2025 20:15:08" │ "1,4 Md o" │ "150 M o" │ "0 o"  │ "86 M o"  │ "7,5 Md o" │ "8,4 Md o" │ "4 Md o" │ "47 %"   │ "52 %"  │ "70 %"  │
│     1 │ "25/07/2025 20:30:06" │ "1,5 Md o" │ "151 M o" │ "0 o"  │ "84 M o"  │ "7,5 Md o" │ "8,4 Md o" │ "4 Md o" │ "60 %"   │ "61 %"  │ "26 %"  │
│     2 │ "25/07/2025 20:45:05" │ "1,5 Md o" │ "151 M o" │ "0 o"  │ "93 M o"  │ "7,5 Md o" │ "8,4 Md o" │ "4 Md o" │ "46 %"   │ "40 %"  │ "59 %"  │
│     3 │ "25/07/2025 21:00:01" │ "1,4 Md o" │ "151 M o" │ "0 o"  │ "69 M o"  │ "7,5 Md o" │ "8,4 Md o" │ "4 Md 